In [ ]:
import os, sys, glob
from pathlib import Path

# adds the package path to the Python path to make sure all the local imports work fine 
if os.path.dirname(os.path.dirname(os.path.dirname(os.getcwd()))) not in sys.path:
    sys.path.append(os.path.dirname(os.path.dirname(os.path.dirname(os.getcwd()))))

# local imports 
from wp4.constants import POLLUTANTS, DATA_DIR_CAMS_AN, DATA_DIR_MERA, DATA_DIR_ERA5, DB_HOST, DB_NAME, DB_USER, DB_PASS
from wp4.baseline.spatial import calculate_wind_speed_direction
from wp4.visualisation.wind import create_windrose

# import remaining packages needed for the script
import xarray as xr
import pandas as pd
import psycopg2 
from datetime import datetime, timedelta, date

import cartopy.crs as ccrs
import matplotlib.pyplot as plt

import plotly.io as pio
import plotly.graph_objects as go
from plotly.subplots import make_subplots
import plotly.express as px

## Load fire events from the database that were detected by satellite and have a FRP value

In [ ]:
conn = psycopg2.connect(dbname=DB_NAME, user=DB_USER, password=DB_PASS, host=DB_HOST)
cur = conn.cursor()

query = """
    SELECT id, datetime, ST_X(geometry), ST_Y(geometry), source, location, reference, type, info, frp
    FROM public.all_fire_events
    WHERE "frp" > 0 AND "frp" IS NOT NULL AND source = 'Satellite'
"""

df_fire_events = pd.read_sql_query(query,con=conn).rename(columns = {'st_x':'longitude', 'st_y':'latitude'})

conn.close()

if len(df_fire_events) == 0:
    print('No fire events found matching your search parameters')
else:
    print(f'Fire events loaded from database: {len(df_fire_events)}')

## Load the ERA5 & MERA datasets

In [ ]:
ds_era5 = xr.open_dataset(Path(DATA_DIR_ERA5).joinpath('era5.nc'))
ds_mera = xr.open_dataset(Path(DATA_DIR_MERA).joinpath('mera.nc'))

## Calculate the wind direction & speed at the time and location of the fire event

In [ ]:
df_fe_data = pd.DataFrame(columns=['wind_direction', 'wind_speed', 'u10'])
ireland_data = []

for ind, row in df_fire_events.iterrows():
    
    if row['datetime'].date() > date(year=2019, month=8, day=31):
        ds_meteo = ds_era5.sel(
            time=row['datetime'].replace(minute=0),
            method='nearest',
        )
        
    else:
        ds_meteo = ds_mera.sel(
            time=row['datetime'].replace(minute=0),
            method='nearest',
        )
        
        ds_meteo['longitude'] = ds_meteo.longitude 
        
    ds_meteo = calculate_wind_speed_direction(ds_meteo, convention='from')
    
    ds_meteo_loc = ds_meteo.sel(
        latitude=row['latitude'],
        longitude=row['longitude'],
        method='nearest'
    )

    df_fe = ds_meteo_loc.to_pandas()[['wind_direction', 'wind_speed', 'u10']]
    df_fe_data.loc[df_fe_data.shape[0]] = [
        df_fe['wind_direction']._magnitude,
        df_fe['wind_speed']._magnitude,
        df_fe['u10'],
    ]
    
    df_ire = ds_meteo.to_dataframe()[['wind_direction', 'wind_speed', 'u10']].reset_index().drop(columns=['longitude', 'latitude'])
    ireland_data += [df_ire]

## Create windrose plots visualizing the wind characteristics for all the fire events

In [ ]:
fig = create_windrose(df_fe_data, title=f'Wind characteristics at location of fire events ({len(df_fire_events)} fires)')
fig.show()

df_ire_combined = pd.concat(ireland_data).reset_index().drop(columns=['index'])
fig = create_windrose(df_ire_combined, title='Wind info for the whole of Ireland during the fire events')
fig.show()

## Create a windrose plot for all the wind information available in the MERA dataset

In [ ]:
ds_mera_wind = calculate_wind_speed_direction(ds_mera, convention='from')
df_mera = ds_mera_wind.to_dataframe()[['wind_direction', 'wind_speed', 'u10']].reset_index().drop(columns=['longitude', 'latitude'])
fig = create_windrose(df_mera, title='Windrose Ireland 2015-2019 (MERA)')
fig.show()